#### Greedy Set Cover $O(log n)$ - approximation algorithm

Given elements from universe $U$ and family of subsets $F$, with each $S \in F$ having a weight $w(S)$:

* initialize $R \leftarrow U$ (remaining uncovered elements)
* while $R$ is not empty:
    * greedily pick the set $S$ with minimum average cost $c_u$ per new element covered, $c_u = \frac{w(S)}{|S \cap R|}$ $\forall u \in S \cap R$ (where $S \cap R$ is the set of new elements covered by set $S$)
    * $R \leftarrow R \setminus (S \cap R)$

In [1]:
import math
import random
import pulp
import time

In [2]:
# this is just a quick and dirty implementation and is not very efficient
def greedy_set_cover(U,F, weights):
    # initialize remaining elements to be covered
    R = set(U)
    cover = []
    # create a copy of the list of subsets
    subsets = list(zip(F, weights))    

    # greedy iterations
    while R:
        # select the min-cost subset
        min_cost = float('inf')
        Smin = None
        wmin = None
        for S, w in subsets:
            cost = w / len(S & R)
            if cost < min_cost:
                min_cost = cost
                Smin = S
                wmin=w

        # add the min-cost subset to the cover
        cover.append((Smin, wmin))

        # remove Smin from the list of subsets
        subsets = [(S, w) for S, w in subsets if S != Smin]

        # remove all new elements covered by Smin from R and from the remaining subsets
        S_new = Smin & R
        R = R - S_new  
        subsets = [(S-S_new, w) for S, w in subsets]


    total_weight = sum([w for S, w in cover])

    return cover, total_weight    

In [3]:
# example universe
U = {1,2,3,4,5,6,7,8}
# example family of sets
eps = 0.1
F = [{1,3,5,7}, {2,4,6,8}, {1}, {2}, {3,4}, {5,6,7,8}]
weights = [1+eps, 1+eps, 1, 1, 1, 1]

exact_cover_idx = [0, 1]
exact_total_weight = sum([weights[idx] for idx in exact_cover_idx])
print(f"Optimal cover: {[F[idx] for idx in exact_cover_idx]}")
print(f"Optimal cover total weight: {exact_total_weight}")


Optimal cover: [{1, 3, 5, 7}, {8, 2, 4, 6}]
Optimal cover total weight: 2.2


In [4]:
cover, total_weight = greedy_set_cover(U,F,weights)
approximation_factor = total_weight/exact_total_weight

print(f" Greedy total weight = {total_weight}, greedy approximation factor = {approximation_factor:.2f} | theoretical approximation factor = 1+log(n) = {1+math.log(len(U)):.2f}")

 Greedy total weight = 4, greedy approximation factor = 1.82 | theoretical approximation factor = 1+log(n) = 3.08


#### A faster implementation of greedy set cover. 

We will use a priority queue to rank the sets in terms of their average cost/price of covering each uncovered element in that set. We will also maintain an inverted index that maps each element to indices of all sets containing that element. The inverted index will allow us to efficiently update the priorities of all other sets containing the newly covered elements on each greedy iteration. 

In [5]:
class PriorityQueue:
    def __init__(self, capacity=10, items=[]):
        self.empty = (0,'Null')
        if len(items)>0:
            assert capacity >= len(items), "capacity must be greater than or equal to the number of items!"
            self.N = len(items)
            self.queue = [self.empty] + items + [self.empty] * (capacity-self.N)
            self.positions = {item[1]:i for i,item in enumerate(self.queue) if item != self.empty}
            # heapify
            self.heapify()
        else:    
            self.queue = [self.empty] * (capacity+1) 
            self.N = 0

    def heapify(self):
        # start from the last parent node up to root
        for i in range(self.N//2, 0, -1):
            # apply heapify down
            self.heapify_down(i)

    def heapify_up(self, i):
        if i > 1:
            current_item = self.queue[i]
            parent_item = self.queue[i//2]
            if current_item[0] < parent_item[0]:
                # swap with parent
                self.queue[i//2] = current_item
                self.queue[i] = parent_item
                # update positions of swapped items
                self.positions[self.queue[i//2][1]] = i//2
                self.positions[self.queue[i][1]] = i
                # recurse from parent position
                self.heapify_up(i//2) 

    def heapify_down(self, i):
        if 2*i > self.N:
            return 
        elif 2*i < self.N:    
            # get index of child with the smaller key
            if self.queue[2*i][0] <= self.queue[2*i+1][0]:
                j = 2*i
            else:
                j= 2*i+1    
        else:
            j = 2*i
        current_item = self.queue[i]
        if current_item[0] > self.queue[j][0]:
            # swap with smallest child
            self.queue[i] = self.queue[j]
            self.queue[j] = current_item
            # update positions of swapped items
            self.positions[self.queue[i][1]] = i
            self.positions[self.queue[j][1]] = j
            # recurse from child position
            self.heapify_down(j)

    def insert(self, item):
        # insert item into first unoccupied slot
        self.queue[self.N+1] = item
        self.positions[item[1]] = self.N+1
        # heapify up
        self.heapify_up(self.N+1)        
        self.N += 1

    def extract_min(self):
        # swap root with the last item
        min_item = self.queue[1]
        self.positions[min_item[1]] = -1 # set position of removed item to -1
        self.queue[1] = self.queue[self.N]
        self.queue[self.N] = self.empty
        self.N -= 1
        # heapify down
        self.heapify_down(1)
        return min_item

    def delete(self, i):
        current_item = self.queue[i]
        self.positions[current_item[1]] = -1 # set position of removed item to -1
        # swap with last item
        self.queue[i] = self.queue[self.N]
        self.queue[self.N] = self.empty
        self.N -= 1
        # check if we need to heapify up or down
        if current_item[0] < self.queue[i//2][0]:
            self.heapify_up(i)
        else:
            self.heapify_down(i)
        return current_item

    def update_key(self, item_id, new_key):
        # get current position of item 
        i = self.positions[item_id]
        if i == -1:
            # item not in priority queue
            return
        (old_key, _) = self.queue[i]
        # update the item key
        self.queue[i] = (new_key, item_id)
        # check if we need to heapify up or down
        if new_key < old_key:
            self.heapify_up(i)
        else:
            self.heapify_down(i)

    def __str__(self):
        print(f"N = {self.N}")
        return str(self.queue)  

In [6]:
def create_inverted_index(U, F):
    inverted_index = {u:[] for u in U}
    for idx, S in enumerate(F):
        for u in S:
            inverted_index[u].append(idx)
    return inverted_index

# this is just a quick and dirty implementation and is not very efficient
def greedy_set_cover_fast(U, F, weights):
    # initialize empty set of covered elements
    C = set()
    cover = []
    # create a copy of the list of subsets
    subsets = list(F)    

    # create a min heap on the subsets using the cost as the key
    pq = PriorityQueue(capacity=len(subsets), items=[(weights[i]/len(F[i]), i) for i in range(len(F))])
    # create inverted index
    inverted_index = create_inverted_index(U, F)
    # keep tranck of number of uncovered elements in each subset
    uncovered = [len(F[i]) for i in range(len(F))]

    # greedy iterations
    while len(C) < len(U):
        # select the min-cost subset
        cost, S_idx = pq.extract_min()
        #print(f"Selected subset index {S_idx} with cost {cost}")
        
        # add the min-cost subset to the cover
        cover.append(S_idx)

        # add newly covered elements to C
        S_new = F[S_idx] - C
        C = C.union(S_new)

        # update number of uncovered elements in each subset that contains newly covered elements
        S_contains_new = set()
        for u in S_new:
            for idx in inverted_index[u]:
                uncovered[idx] -= 1
                S_contains_new.add(idx)
        
        # update costs of all subsets that contain newly covered elements of S_idx in priority queue
        for idx in S_contains_new:
            if uncovered[idx] > 0:
                pq.update_key(idx, weights[idx]/uncovered[idx])
            else:
                pq.update_key(idx, float('inf'))

    total_weight = sum([weights[idx] for idx in cover])

    return cover, total_weight    

In [7]:
# example universe
U = {1,2,3,4,5,6,7,8}
# example family of sets
eps = 0.1
F = [{1,3,5,7}, {2,4,6,8}, {1}, {2}, {3,4}, {5,6,7,8}]
weights = [1+eps, 1+eps, 1, 1, 1, 1]

# greedy cover
cover, total_weight = greedy_set_cover_fast(U, F, weights)
approximation_factor = total_weight/exact_total_weight

print(f" Greedy total weight = {total_weight}, greedy approximation factor = {approximation_factor:.2f} | theoretical approximation factor = 1+log(n) = {1+math.log(len(U)):.2f}")


 Greedy total weight = 4, greedy approximation factor = 1.82 | theoretical approximation factor = 1+log(n) = 3.08


#### Test on larger universes and larger family of random subsets, compare with exact integer program solution

In [8]:
def solve_set_cover_IP(U, F, weights):
    # create IP problem instance
    prob = pulp.LpProblem("Set Covering Problem", pulp.LpMinimize)
    # create binary decision variables
    x = pulp.LpVariable.dicts("x", range(len(F)), lowBound=0, upBound=1, cat=pulp.LpInteger)
    # define objective function
    prob += pulp.lpSum([weights[i] * x[i] for i in range(len(F))])
    # define constraints (each element in U must be covered)
    for e in U:
        prob += pulp.lpSum([x[i] for i in range(len(F)) if e in F[i]]) >= 1

    # solve the LP
    prob.solve()

    # extract optimal solution
    optimal_subsets_idx = [i for i in range(len(F)) if x[i].value() == 1]
    optimal_weights = [weights[i] for i in range(len(F)) if x[i].value() == 1]   

    return optimal_subsets_idx, optimal_weights


def generate_random_set_cover_instance(n, m):
    U = list(range(1,n+1))
    F = []
    weights = []
    for i in range(m):
        # sample a random subset
        S = set(random.sample(U, random.randint(1, n)))
        F.append(S)
        # assign random weight
        weights.append(random.uniform(1, 10))
    return U, F, weights    

In [9]:
# generate random set cover instance
n = 5000
m = 500
U, F, weights = generate_random_set_cover_instance(n, m)

In [10]:
# get optimal IP solution
start_time = time.time()
optimal_subsets_idx, optimal_weights = solve_set_cover_IP(U, F, weights)
end_time = time.time()
optimal_total_weight = sum(optimal_weights)
print(f"Optimal cover subset ids: {optimal_subsets_idx}")
print(f"Optimal cover total weight: {optimal_total_weight}")
print(f"Time to solve IP: {end_time-start_time:.2f} seconds\n")

# greedy cover
start_time = time.time()
greedy_cover, total_weight = greedy_set_cover_fast(U, F, weights)
end_time = time.time()
approximation_factor = total_weight/optimal_total_weight

print(f"Greedy cover subset ids: {greedy_cover}")
print(f"Greedy cover total weight = {total_weight}, approximation factor = {approximation_factor:.2f} | theoretical approximation factor = 1+log(n) = {1+math.log(len(U)):.2f}")
print(f"Time to solve greedy: {end_time-start_time:.2f} seconds")

/home/tanzid/miniconda3/envs/torch_clone_2/lib/python3.11/site-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/tanzid/miniconda3/envs/torch_clone_2/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/19aefc1129db46c5ac18d3f13ab40408-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/19aefc1129db46c5ac18d3f13ab40408-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 5005 COLUMNS
At line 1279185 RHS
At line 1284186 BOUNDS
At line 1284687 ENDATA
Problem MODEL has 5000 rows, 500 columns and 1272679 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2.06699 - 0.13 seconds
Cgl0004I processed model has 5000 rows, 500 columns (500 integer (500 of which binary)) and 1272679 elements
Cbc0038I Initial state - 24 integers unsatisfied sum - 1.4384
Cbc0038I Pass   1: (9.42 seconds) suminf.    0.00025 (27) obj. 8.47831 iterations 4209
Cbc0038I Solution found of 8.4786
Cbc0038I Bef